In [200]:
import pandas as pd
import os
import matplotlib.pyplot as plt

# Table of Contents

1. [Load data](#Load-data)
2. [Sample Selection Summary](#Summary-of-the-"sample-selection")
3. [Subdivisions](#Subdivisions)

## Load data

In [201]:
html = pd.read_csv('../data/2_final_webpage_metadata.csv')

How many HTMLs have we successfully parsed? Filter out all the ones we couldn't

In [202]:
html = html[(html['is_429'] == 0) & (html['is_parsable'] == 1) & (html['file_exists'] == 1)]
print(len(html.index))

644439


Drop those columns now...

In [203]:
html = html.drop(['is_429', 'is_parsable', 'file_exists'], axis=1)

In [204]:
html = html.drop(columns=['Unnamed: 0'])

How many companies are we working with?

In [205]:
print(html['entityid'].nunique())

44110


Combine data with the input file

In [206]:
in_file = pd.read_csv('../data/startup_url_list.csv')
html = html.merge(in_file, on='entityid')

In [208]:
html.head()

entityid  capture_yr  capture_m                                file_path  \
0  1006526630        2001          3  data/html/1006526630/2001/03/index.html   
1  1006526630        2001          5  data/html/1006526630/2001/05/index.html   
2  1006526630        2002          5  data/html/1006526630/2002/05/index.html   
3  1006526630        2002         11  data/html/1006526630/2002/11/index.html   
4  1006526630        2003          6  data/html/1006526630/2003/06/index.html   

   website_size_kb  careers  blog  login  contact  team  ...  form_count  \
0         4.592773        0     0      0        0     0  ...           1   
1         4.822266        0     0      0        0     0  ...           1   
2         2.606445        0     0      0        0     0  ...           0   
3         2.604492        0     0      0        0     0  ...           0   
4         2.595703        0     0      0        0     0  ...           0   

   script_count  embedded_js  external_js            weburl pb_companyid  \
0             3         True         True  www.usclaims.com          NaN   
1             3         True         True  www.usclaims.com          NaN   
2             3         True         True  www.usclaims.com          NaN   
3             3         True         True  www.usclaims.com          NaN   
4             3         True         True  www.usclaims.com          NaN   

    startdate      lastVC  ownershipstatus exit_date  
0  1996-06-01  2020-01-01  Acquired/Merged       NaN  
1  1996-06-01  2020-01-01  Acquired/Merged       NaN  
2  1996-06-01  2020-01-01  Acquired/Merged       NaN  
3  1996-06-01  2020-01-01  Acquired/Merged       NaN  
4  1996-06-01  2020-01-01  Acquired/Merged       NaN  

[5 rows x 35 columns]

In [209]:
html.dtypes.to_dict()

{'entityid': dtype('int64'),
 'capture_yr': dtype('int64'),
 'capture_m': dtype('int64'),
 'file_path': dtype('O'),
 'website_size_kb': dtype('float64'),
 'careers': dtype('int64'),
 'blog': dtype('int64'),
 'login': dtype('int64'),
 'contact': dtype('int64'),
 'team': dtype('int64'),
 'about': dtype('int64'),
 'news': dtype('int64'),
 'faq': dtype('int64'),
 'call_to_action': dtype('int64'),
 'testimonial': dtype('int64'),
 'title': dtype('O'),
 'description': dtype('O'),
 'keywords': dtype('O'),
 'author': dtype('O'),
 'language': dtype('O'),
 'p_count': dtype('int64'),
 'h_count': dtype('int64'),
 'img_count': dtype('int64'),
 'a_count': dtype('int64'),
 'table_count': dtype('int64'),
 'form_count': dtype('int64'),
 'script_count': dtype('int64'),
 'embedded_js': dtype('O'),
 'external_js': dtype('O'),
 'weburl': dtype('O'),
 'pb_companyid': dtype('O'),
 'startdate': dtype('O'),
 'lastVC': dtype('O'),
 'ownershipstatus': dtype('O'),
 'exit_date': dtype('O')}

## Summary of the "sample selection"

## Subdivisions

For all companies that have survived at least *n* years, divide that into exited and non-exited companies...

Let's start with 5. And then move to 10

Let's add some columns that will enable us to do this easier

In [211]:
html["has_exit"] = (html["exit_date"].isna() == False).astype('int8')
html["end_yr"] = html["exit_date"].str.slice(start=0, stop=4).astype('float' ,errors='ignore').fillna(2023)
html["start_yr"] = html["startdate"].str.slice(start=0, stop=4).fillna(html["lastVC"].str.slice(start=0, stop=4)).astype('float', errors='ignore')
html["lifespan"] = html["end_yr"] - html["start_yr"]

Create a new DF that contains companies that were founded in 2012 and survived at least 5 years

In [226]:
survived_5 = html[(html["lifespan"] > 5) & (html["start_yr"] == 2012)]

Of the companies that survived at least five years, how *many* had an exit? How many didn't?

In [227]:
print(survived_5[survived_5["has_exit"] == 1]['entityid'].nunique())
print(survived_5[survived_5["has_exit"] == 0]['entityid'].nunique())

297
1655


Divide these into two seperate DFs - "exited" and "alive"

In [229]:
s5_exit = survived_5[survived_5["has_exit"] == 1]
s5_alive = survived_5[survived_5["has_exit"] == 0]

Do these companies begin to go in different directions, after, say, year 2?

Am I trying to measure the change in companies *in aggregate* or on a company-by-company basis (i.e. bootstrap)?

Let's just start with "in aggregate", so essentially measuring averages